# Reach Curve Modeling Based On Timespends Data
 
This notebook explores and models reach curve.

## Objectives:
1. Load and explore data
2. Exploratory data analysis
3. Research
3. Reach curve modeling
4. Evaluation method
5. Reach Optimization
6. Recommendations and Insights

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn_extra.cluster import KMedoids

import warnings
warnings.filterwarnings('ignore')

## 1. Load and explore data

In [ ]:
data = pd.read_csv("../data/timespends.csv", index_col=0)

TOTAL_NUM_SAMPLES = len(data)
print(f"No. of samples: {TOTAL_NUM_SAMPLES}")
print(f"Sample:\n {data.head()}")


plt.plot(sorted(data['timespends']))
plt.title("Timespends (sorted)")
plt.show()

### Basic observations

So, the entire population comprises 10,333 samples, for which the average time spent on a particular medium was measured.

The above plot reveals about 20% of sampled population doesn't use the medium at all. This means there's simply no way to reach them through this medium, and it will also set an upper limit for the reach curve (around 80%).

On the other hand, about 15% of the population spends the most time on this medium compared to all other groups. This group will likely contain the highest number of individuals who will see an advertisement multiple times. Because of it - the reach curve will flatten – despite an increase in the number of impressions.

## 2. Exploratory Data Analysis

In [ ]:
data['timespends'].value_counts().head(15)

In [ ]:
N_CLUSTERS = 7
X = data['timespends'].to_numpy().reshape(-1, 1)
kmedoids = KMedoids(n_clusters=N_CLUSTERS, random_state=0, method='pam').fit(X)

In [ ]:
clustered_data = data.copy()
clustered_data['cluster'] = kmedoids.predict(X)
clustered_data['cluster-timespend'] = clustered_data.apply(lambda row: kmedoids.cluster_centers_[int(row['cluster'])][0], axis=1)
clustered_data.head()

In [ ]:
plt.plot(sorted(clustered_data['cluster-timespend']))
plt.title("Timespends (sorted) - simplified")

In [ ]:
df = pd.DataFrame(clustered_data['cluster-timespend'].value_counts())
df.reset_index(inplace=True)
df.sort_values(by='cluster-timespend', inplace=True, ascending=False)
df['perc_of_total'] = df['count'] / TOTAL_NUM_SAMPLES * 100
df['perc_of_total_cum'] = np.cumsum(df['perc_of_total'].to_numpy())
df

### Some observations
1. We have 7 main user groups defined.
2. The maximum value the reach curve will achieve is ```77%```
3. The minimum value is 0 (it would be interesting to estimate this worst-case scenario somehow—i.e., what would be the chances that absolutely no one gets any impression even once).
4. The reach curve is a non-decreasing curve because an impression, once received, cannot be unreceived (or taken back).

However, one can imagine that if a single person receives too many impressions, the curve might not only stop growing, but also grow more slowly – if that person discourages others from using the medium.

### Simplistic scenario
Data observations provide some insight into the approximate shape of the reach curve.

Really simplistic scenario:
There's an initial rapid growth to approximately 14 percent, followed by a similarly swift increase to 31 percent, and then to 52 percent, and so on. This offers a clearer intuition regarding the mathematical model, indicating that ```timespend``` will correspond with the reach curve's rate of growth (its derivative), whereas ```perc_of_total_cum``` will correspond to distinct points where the reach curve exhibits unique changes.

I'd like to start with a simple model, even with some simplifications and additional assumptions (which I'll mention shortly), to have a reference point. After that, I'll try to incorporate more subtle elements into the model, so the assumptions better reflect the actual data I've received.

Of course, in this simplified scenario, I'm assuming that people with the highest timespend (those with 0.84) will get impression first, followed by those in the next group (0.67), and so on.

### A little more serious approach
I've been thinking about this more, and I could simplify the situation for a moment by observing a group that spends a similar amount of time watching the medium. I would then consider a random variable X_i​, which assigns to each user i the number of viewed impressions. So, for the entire group, I would be dealing with a vector of random variables, for which several precise questions could be posed. Each of the individual components of this vector could follow a Bernoulli distribution with a fixed probability of success p, which would depend on the frequency of viewing that particular medium.

In [ ]:
splines = list()

start = 0.0
for _, row in df.iterrows():
    if row['cluster-timespend'] == 0:
        break
    stop = row['perc_of_total_cum']
    splines.append(np.arange(start=start, stop=stop, step=row['cluster-timespend']))
    start = stop

dummy_reach_curve = np.concatenate(splines)
plt.plot(dummy_reach_curve)

## 3. Research

### Articles
1. [Reach Measurement, Optimization and Frequency Capping In Targeted Online Advertising Under k-Anonymity](https://arxiv.org/pdf/2501.04882v1)
2. [Estimating reach curves from one data point](https://static.googleusercontent.com/media/research.google.com/en//pubs/archive/43218.pdf)

I chose these articles because they seemed interesting and were the easiest to find with the browser. I wanted to familiarize myself a bit more with the way of thinking and the general problems associated with modeling reach curves, as well as the methods specialists use. It looks quite interesting!

## 4. Reach Curve Modeling

In [ ]:
def reach_curve(impressions: float) -> float:
    raise NotImplementedError

## 5. Evaluation method

## 6. Reach Optimization

## 7. Recommendations and Insights

Based on the reach curve analysis:

1. **Optimal impression levels**: The marginal reach decreases as impressions increase
2. **Cost efficiency**: There's a sweet spot for cost-efficient reach
3. **Model selection**: Choose the model with better fit for your data

### Next Steps:
- Segment analysis by campaign type
- Time-series analysis of reach patterns
- Multi-channel reach optimization